In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

data = pd.read_csv('data.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day_of_week'] = data['Date'].dt.dayofweek
data['Day'] = data['Date'].dt.day
ph = data['Public Holiday'].copy()
data['Public Holiday'] = data['Public Holiday'].fillna(0)
data['Public Holiday'] = data['Public Holiday'].apply(lambda x: 1 if x else 0)
data['Event'] = data['Event'].apply(lambda x: 1 if x == 'TRUE' else 0)

X = data[['Day_of_week', 'Day', 'Month', 'Year','Event', 'Public Holiday',
          'India_Reservation']]
y = data[['Customers_Chinese', 'Customers_India']]

busy_threshold = 100
def is_busy(customers_prediction):
    return customers_prediction > busy_threshold

X_train,X_test,y_train, y_test = train_test_split(X,y,test_size = 7)

model = DecisionTreeRegressor(random_state=42)
model.fit(X_train,y_train)

predictions = pd.DataFrame(model.predict(X_test), columns=['Predicted_Customers_Chinese', 'Predicted_Customers_India'])
X_test['Date'] = X_test.apply(lambda row: f"{row['Day']}/{row['Month']}/{row['Year']}", axis=1)
predictions['Date']= pd.to_datetime(X_test['Date'].values, format='%d/%m/%Y')
predictions['Day'] = predictions['Date'].dt.day_name()
X_test.reset_index(drop=True, inplace=True)
predictions['Public Holiday'] = ph
predictions['India_Reservation'] = X_test['India_Reservation']
predictions['Chinese_Buffet_Busy'] = predictions['Predicted_Customers_Chinese'].apply(is_busy)
predictions['Indian_Buffet_Busy'] = predictions['Predicted_Customers_India'].apply(is_busy)
predictions = predictions[['Date', 'Day', 'Public Holiday', 'Predicted_Customers_Chinese', 'Chinese_Buffet_Busy', 'India_Reservation','Predicted_Customers_India', 'Indian_Buffet_Busy']]

print(predictions)
